# Record Linkage

In [83]:
import recordlinkage
import pandas as pd
import collections
import time
import os

In [84]:
PATH_DS_L = './Mediated Datasets/cbinsights_DDD_m.jsonl'
PATH_DS_R = './Mediated Datasets/cbinsights_iGMM_m.jsonl'

In [25]:
df_l = pd.read_json(PATH_DS_L, encoding='utf-8', lines=True, dtype=object)
df_l

,name,country,industry,founded
0,lacework,united states,cybersecurity,2015
1,tipalti,united states,fintech,2010
2,tempus,united states,health,2015
3,anduril,united states,artificial intelligence,2017
4,bolt,estonia,auto & transportation,2013
...,...,...,...,...
1180,spacex,united states,other,2002
1181,fanatics,united states,e-commerce & direct-to-consumer,1995
1182,instacart,united states,"supply chain, logistics, & delivery",2012
1183,databricks,united states,data management & analytics,2013


In [55]:
df_r = pd.read_json(PATH_DS_R, encoding='utf-8', lines=True, dtype=object)
df_r

,name,country,industry
0,bytedance,china,artificial intelligence
1,spacex,united states,other
2,shein,china,e-commerce & direct-to-consumer
3,stripe,united states,fintech
4,canva,australia,internet software & services
...,...,...,...
1197,leadsquared,india,internet software & services
1198,fourkites,united states,"supply chain, logistics, & delivery"
1199,vulcanforms,united states,"supply chain, logistics, & delivery"
1200,singlestore,united states,data management & analytics


In [85]:
def get_features(df_l, df_r, name_threshold, is_name):
    feature_value = 0
    columns_l = df_l.columns.values.tolist()
    columns_r = df_r.columns.values.tolist()
    # set_index
    indexer = recordlinkage.Index()
    indexer.full()
    candidate_links = indexer.index(df_l, df_r)

    # Comparison step
    compare_cl = recordlinkage.Compare()
    compare_cl.string("name", "name", method="levenshtein", threshold=name_threshold, label="name")

    if not is_name:
        if 'ceo' in columns_l and 'ceo' in columns_r:
            compare_cl.string("ceo", "ceo", method="levenshtein", threshold=0.85, label="ceo")
            feature_value = 1

    # compare_cl.exact("country", "country", label="country")

    return compare_cl.compute(candidate_links, df_l, df_r), feature_value


def get_pairs(features, feature_value):
    # Classification step
    pairs = features[features.sum(axis=1) > feature_value]
    return pairs['name'].keys().to_list()


def rename_columns(columns_l, columns_r):
    columns_l = [c + '_l' for c in columns_l]
    columns_r = [c + '_r' for c in columns_r]
    columns_join = columns_l + columns_r
    return columns_l, columns_r

def remove_suffix(columns):
    return [c[:-2] for c in columns]


def find_duplicated_columns(columns_l, columns_r):
    attr_cleaned = columns_l + columns_r
    duplicates = [item for item, count in collections.Counter(attr_cleaned).items() if count > 1]
    return duplicates

# def join(matching_dataset_left, matching_dataset_right):
#     column_left, column_right = rename_columns(matching_dataset_left.columns.values.tolist(),
#                                            matching_dataset_right.columns.values.tolist())
#     matching_dataset_left.columns = column_left
#     matching_dataset_right.columns = column_right
#     joined_df = pd.concat([matching_dataset_left, matching_dataset_right], axis=1)
#
#     duplicates = find_duplicated_columns(df_l.columns.values.tolist(), df_r.columns.values.tolist())
#     for col in duplicates:
#         candidates = joined_df[[col + '_l', col + '_r']].sample(20)
#         candidate_labels = candidates.columns.values.tolist()
#         display(candidates.sample(10))
#         time.sleep(1)
#         idx_drop = int(input("Inserisci l'indice (0 o 1) della colonna che vuoi scartare nel dataset finale: "))
#         joined_df.drop(candidate_labels[idx_drop], axis=1, inplace=True)
#
#     joined_df.columns = [col[:-2] for col in joined_df.columns.values.tolist()]
#     return joined_df

def join(matching_dataset_left, matching_dataset_right):
    duplicates = find_duplicated_columns(matching_dataset_left.columns.values.tolist(), matching_dataset_right.columns.values.tolist())

    column_left, column_right = rename_columns(matching_dataset_left.columns.values.tolist(),
                                           matching_dataset_right.columns.values.tolist())
    matching_dataset_left.columns = column_left
    matching_dataset_right.columns = column_right
    joined_df = pd.concat([matching_dataset_left, matching_dataset_right], axis=1)

    # duplicates = find_duplicated_columns(matching_dataset_left.columns.values.tolist(), matching_dataset_right.columns.values.tolist())
    for col in duplicates:
        candidates = joined_df[[col + '_l', col + '_r']]
        candidate_labels = candidates.columns.values.tolist()
        #display(candidates.sample(10))
        time.sleep(1)
        idx_drop = 0 #int(input("Inserisci l'indice (0 o 1) della colonna che vuoi scartare nel dataset finale: "))
        joined_df.drop(candidate_labels[idx_drop], axis=1, inplace=True)

    joined_df.columns = [col[:-2] for col in joined_df.columns.values.tolist()]
    return joined_df


def merge_datasets(df_l, df_r):
    features, feature_value = get_features(df_l, df_r, 0.95, False)
    pairs = get_pairs(features, feature_value)
    features.sum(axis=1).value_counts().sort_index(ascending=False)

    left = []
    right = []

    for elem in pairs:
        left.append(df_l.loc[elem[0]])
        right.append(df_r.loc[elem[1]])

    matching_dataset_left = pd.DataFrame(left)
    matching_dataset_right = pd.DataFrame(right)
    matching_dataset_left = matching_dataset_left.reset_index(drop=True)
    matching_dataset_right = matching_dataset_right.reset_index(drop=True)

    difference_l = pd.concat([df_l, matching_dataset_left]).drop_duplicates(keep=False)
    difference_r = pd.concat([df_r, matching_dataset_right]).drop_duplicates(keep=False)

    match_join_df = join(matching_dataset_left, matching_dataset_right)
    display(match_join_df)



    features_difference, feature_value_difference = get_features(difference_l, difference_r, 0.95, True)
    pairs_difference = get_pairs(features_difference, feature_value_difference)
    features_difference.sum(axis=1).value_counts().sort_index(ascending=False)

    left = []
    right = []

    for elem in pairs_difference:
        left.append(difference_l.loc[elem[0]])
        right.append(difference_r.loc[elem[1]])

    matching_difference_dataset_left = pd.DataFrame(left)
    matching_difference_dataset_right = pd.DataFrame(right)
    matching_difference_dataset_left = matching_difference_dataset_left.reset_index(drop=True)
    matching_difference_dataset_right = matching_difference_dataset_right.reset_index(drop=True)

    difference_join_df = join(matching_difference_dataset_left, matching_difference_dataset_right)
    display(difference_join_df)

    result= pd.concat([match_join_df, difference_join_df])
    display(result)

    matching_difference_dataset_left.columns = remove_suffix(matching_difference_dataset_left.columns.values.tolist())
    matching_difference_dataset_right.columns = remove_suffix(matching_difference_dataset_right.columns.values.tolist())

    unique_left_df =pd.concat([difference_l, matching_difference_dataset_left]).drop_duplicates(keep=False)
    result = pd.concat([result, unique_left_df])

    unique_right_df = pd.concat([difference_r, matching_difference_dataset_right]).drop_duplicates(keep=False)
    result = pd.concat([result, unique_right_df])

    result = result.reset_index(drop=True)
    
    return result

In [57]:
result = merge_datasets(df_l, df_r)
print("\n*\n*\n*")
display(result)

,name_l,name_r
101,iodine software,iodine software
1109,svolt,svolt
279,hmd global,hmd global
104,nobroker,nobroker
860,cribl,cribl
526,hive box,hive box
920,csurgical,csurgical
343,turing,turing
1173,revolut,revolut
1122,argo ai,argo ai


Inserisci l'indice (0 o 1) della colonna che vuoi scartare nel dataset finale: 0


,country_l,country_r
5,estonia,estonia
663,united states,united states
908,vietnam,vietnam
1054,united states,united states
650,china,china
859,united states,united states
719,united states,united states
251,china,china
1035,france,france
340,canada,canada


Inserisci l'indice (0 o 1) della colonna che vuoi scartare nel dataset finale: 0


,industry_l,industry_r
442,fintech,fintech
85,travel,travel
967,hardware,hardware
732,health,health
40,fintech,fintech
985,internet software & services,internet software & services
1090,e-commerce & direct-to-consumer,e-commerce & direct-to-consumer
246,artificial intelligence,artificial intelligence
239,mobile & telecommunications,mobile & telecommunications
274,e-commerce & direct-to-consumer,e-commerce & direct-to-consumer


...

,founded,name,country,industry
0,2015,lacework,united states,cybersecurity
1,2010,tipalti,united states,fintech
2,2015,tempus,united states,health
3,2017,anduril,united states,artificial intelligence
4,2013,bolt,united states,fintech
...,...,...,...,...
1178,2002,spacex,united states,other
1179,1995,fanatics,united states,e-commerce & direct-to-consumer
1180,2012,instacart,united states,"supply chain, logistics, & delivery"
1181,2013,databricks,united states,data management & analytics


""


,founded,name,country,industry
0,2015,lacework,united states,cybersecurity
1,2010,tipalti,united states,fintech
2,2015,tempus,united states,health
3,2017,anduril,united states,artificial intelligence
4,2013,bolt,united states,fintech
...,...,...,...,...
1178,2002,spacex,united states,other
1179,1995,fanatics,united states,e-commerce & direct-to-consumer
1180,2012,instacart,united states,"supply chain, logistics, & delivery"
1181,2013,databricks,united states,data management & analytics



*
*
*


,founded,name,country,industry
0,2015,lacework,united states,cybersecurity
1,2010,tipalti,united states,fintech
2,2015,tempus,united states,health
3,2017,anduril,united states,artificial intelligence
4,2013,bolt,united states,fintech
...,...,...,...,...
1044,NaN,printful,united states,other
1065,NaN,xiaobing,china,fintech
1078,NaN,dunamu,south korea,fintech
1091,NaN,pandadoc,united states,internet software & services


In [61]:
df_l = result.reset_index()
df_r = pd.read_json("./Mediated Datasets/disfold_Avengers_m.jsonl", encoding='utf-8', lines=True, dtype=object)

In [62]:
result = merge_datasets(result, df_r)
print("\n*\n*\n*")
display(result)

ValueError: index of DataFrame is not unique

In [86]:
SOURCE_PATH = "./Mediated Datasets/"

In [87]:
files = []
for entry in os.listdir(SOURCE_PATH):
    if os.path.isfile(os.path.join(SOURCE_PATH, entry)):
        files.append(entry)
        
print(files)

['cbinsights_DDD_m.jsonl', 'cbinsights_iGMM_m.jsonl', 'companiesMarketCap_Avengers_m.jsonl']


In [88]:
res_temp = pd.DataFrame()
res_temp = pd.read_json(SOURCE_PATH + files[0], encoding='utf-8', lines=True, dtype=object)
for i in range(len(files)-1):
    dd = pd.read_json(SOURCE_PATH + files[i+1], encoding='utf-8', lines=True, dtype=object)
    print("res_temp:")
    display(res_temp)
    print("dd:")
    display(dd)
    res_temp = merge_datasets(res_temp, dd)
    display(res_temp)
    
print("Finito!")
display(res_temp)

res_temp:


,name,country,industry,founded
0,lacework,united states,cybersecurity,2015
1,tipalti,united states,fintech,2010
2,tempus,united states,health,2015
3,anduril,united states,artificial intelligence,2017
4,bolt,estonia,auto & transportation,2013
...,...,...,...,...
1180,spacex,united states,other,2002
1181,fanatics,united states,e-commerce & direct-to-consumer,1995
1182,instacart,united states,"supply chain, logistics, & delivery",2012
1183,databricks,united states,data management & analytics,2013


dd:


,name,country,industry
0,bytedance,china,artificial intelligence
1,spacex,united states,other
2,shein,china,e-commerce & direct-to-consumer
3,stripe,united states,fintech
4,canva,australia,internet software & services
...,...,...,...
1197,leadsquared,india,internet software & services
1198,fourkites,united states,"supply chain, logistics, & delivery"
1199,vulcanforms,united states,"supply chain, logistics, & delivery"
1200,singlestore,united states,data management & analytics


,founded,name,country,industry
0,2015,lacework,united states,cybersecurity
1,2010,tipalti,united states,fintech
2,2015,tempus,united states,health
3,2017,anduril,united states,artificial intelligence
4,2013,bolt,united states,fintech
...,...,...,...,...
1178,2002,spacex,united states,other
1179,1995,fanatics,united states,e-commerce & direct-to-consumer
1180,2012,instacart,united states,"supply chain, logistics, & delivery"
1181,2013,databricks,united states,data management & analytics


...

""


,founded,industry,name,market_cap,country,share_price,categories
0,2017,fintech,dana,2420000000,usa,16,
1,2007,health,prime medicine,1600000000,usa,16,"pharmaceuticals, biotech, genomics, gene therapy"
2,2020,health,eqrx,1210000000,usa,2,"pharmaceuticals, biotech"
3,2013,fintech,momo,2029999999,china,10,internet


,founded,industry,name,market_cap,country,share_price,categories
0,2017,fintech,dana,2420000000.0,usa,16.0,
1,2007,health,prime medicine,1600000000.0,usa,16.0,"pharmaceuticals, biotech, genomics, gene therapy"
2,2020,health,eqrx,1210000000.0,usa,2.0,"pharmaceuticals, biotech"
3,2013,fintech,momo,2029999999.0,china,10.0,internet
4,2015,cybersecurity,lacework,NaN,united states,NaN,NaN
...,...,...,...,...,...,...,...
7104,NaN,NaN,neogen,4240000000,usa,19,
7105,NaN,NaN,pnm resources,4240000000,usa,49,
7106,NaN,NaN,kinnevik,4250000000,sweden,15,investment
7107,NaN,NaN,topgolf callaway brands,4250000000,usa,23,sports goods


Finito!


,founded,industry,name,market_cap,country,share_price,categories
0,2017,fintech,dana,2420000000.0,usa,16.0,
1,2007,health,prime medicine,1600000000.0,usa,16.0,"pharmaceuticals, biotech, genomics, gene therapy"
2,2020,health,eqrx,1210000000.0,usa,2.0,"pharmaceuticals, biotech"
3,2013,fintech,momo,2029999999.0,china,10.0,internet
4,2015,cybersecurity,lacework,NaN,united states,NaN,NaN
...,...,...,...,...,...,...,...
7104,NaN,NaN,neogen,4240000000,usa,19,
7105,NaN,NaN,pnm resources,4240000000,usa,49,
7106,NaN,NaN,kinnevik,4250000000,sweden,15,investment
7107,NaN,NaN,topgolf callaway brands,4250000000,usa,23,sports goods


In [29]:
unique_left_df =pd.concat([difference_l, matching_difference_dataset_left]).drop_duplicates(keep=False)
result = pd.concat([result, unique_left_df])

In [30]:
result

,name,country,industry,founded
0,lacework,united states,cybersecurity,2015
1,tipalti,united states,fintech,2010
2,tempus,united states,health,2015
3,anduril,united states,artificial intelligence,2017
4,bolt,estonia,auto & transportation,2013
...,...,...,...,...
1180,spacex,united states,other,2002
1181,fanatics,united states,e-commerce & direct-to-consumer,1995
1182,instacart,united states,"supply chain, logistics, & delivery",2012
1183,databricks,united states,data management & analytics,2013


In [31]:
unique_right_df = pd.concat([difference_r, matching_difference_dataset_right]).drop_duplicates(keep=False)
result = pd.concat([result, unique_right_df])

In [32]:
result

,name,country,industry,founded,employees,ceo
0,lacework,united states,cybersecurity,2015,NaN,NaN
1,tipalti,united states,fintech,2010,NaN,NaN
2,tempus,united states,health,2015,NaN,NaN
3,anduril,united states,artificial intelligence,2017,NaN,NaN
4,bolt,estonia,auto & transportation,2013,NaN,NaN
...,...,...,...,...,...,...
894,markel corporation,NaN,NaN,1930,"20,300",thomas sinnickson gayner
895,brown & brown inc,NaN,NaN,1939,"12,023",j powell brown cpcu cpcu
896,nomura research institute ltd,NaN,NaN,,"16,196",shingo konomoto
897,mgm resorts international,NaN,NaN,1986,"52,000",william joseph hornbuckle iv


In [653]:
features, feature_value = get_features(df_l, df_r, 0.95, False)
pairs = get_pairs(features, feature_value)

In [654]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

0.0    1065315
dtype: int64

In [656]:
left = []
right = []

for elem in pairs:
    left.append(df_l.loc[elem[0]])
    right.append(df_r.loc[elem[1]])

matching_dataset_left = pd.DataFrame(left)
matching_dataset_right = pd.DataFrame(right)
matching_dataset_left = matching_dataset_left.reset_index(drop=True)
matching_dataset_right = matching_dataset_right.reset_index(drop=True)

In [657]:
matching_dataset_left

""


In [3]:
def get_features(df_l, df_r, name_threshold, is_name):
    feature_value = 0
    columns_l = df_l.columns.values.tolist()
    columns_r = df_r.columns.values.tolist()
    # set_index
    indexer = recordlinkage.Index()
    indexer.full()
    candidate_links = indexer.index(df_l, df_r)

    # Comparison step
    compare_cl = recordlinkage.Compare()
    compare_cl.string("name", "name", method="levenshtein", threshold=name_threshold, label="name")

    if not is_name:
        if 'ceo' in columns_l and 'ceo' in columns_r:
            compare_cl.string("ceo", "ceo", method="levenshtein", threshold=0.85, label="ceo")
            feature_value = 1

    # compare_cl.exact("country", "country", label="country")

    return compare_cl.compute(candidate_links, df_l, df_r), feature_value


def get_pairs(features, feature_value):
    # Classification step
    pairs = features[features.sum(axis=1) > feature_value]
    return pairs['name'].keys().to_list()


def rename_columns(columns_l, columns_r):
    columns_l = [c + '_l' for c in columns_l]
    columns_r = [c + '_r' for c in columns_r]
    columns_join = columns_l + columns_r
    return columns_l, columns_r


def find_duplicated_columns(columns_l, columns_r):
    attr_cleaned = columns_l + columns_r
    duplicates = [item for item, count in collections.Counter(attr_cleaned).items() if count > 1]
    return duplicates

def join(matching_dataset_left, matching_dataset_right):
    column_left, column_right = rename_columns(matching_dataset_left.columns.values.tolist(),
                                           matching_dataset_right.columns.values.tolist())
    matching_dataset_left.columns = column_left
    matching_dataset_right.columns = column_right
    joined_df = pd.concat([matching_dataset_left, matching_dataset_right], axis=1)

    duplicates = find_duplicated_columns(df_l.columns.values.tolist(), df_r.columns.values.tolist())
    for col in duplicates:
        candidates = joined_df[[col + '_l', col + '_r']].sample(20)
        candidate_labels = candidates.columns.values.tolist()
        display(candidates.sample(10))
        time.sleep(1)
        idx_drop = int(input("Inserisci l'indice (0 o 1) della colonna che vuoi scartare nel dataset finale: "))
        joined_df.drop(candidate_labels[idx_drop], axis=1, inplace=True)

    joined_df.columns = [col[:-2] for col in joined_df.columns.values.tolist()]
    return joined_df

In [658]:
matching_dataset_right

""


In [659]:
difference_l = pd.concat([df_l, matching_dataset_left]).drop_duplicates(keep=False)
difference_r = pd.concat([df_r, matching_dataset_right]).drop_duplicates(keep=False)

In [660]:
difference_l

,name,country,industry,founded
0,lacework,united states,cybersecurity,2015
1,tipalti,united states,fintech,2010
2,tempus,united states,health,2015
3,anduril,united states,artificial intelligence,2017
4,bolt,estonia,auto & transportation,2013
...,...,...,...,...
1180,spacex,united states,other,2002
1181,fanatics,united states,e-commerce & direct-to-consumer,1995
1182,instacart,united states,"supply chain, logistics, & delivery",2012
1183,databricks,united states,data management & analytics,2013


In [661]:
difference_r

,founded,employees,ceo,name
0,"april 1, 1976","100,000",timothy d cook,apple
1,"april 4, 1975","181,000",satya nadella,microsoft
2,,,amin h nasser,aramco
3,"october 2, 2015","156,500",sundar pichai,alphabet
4,"july 5, 1994","1,608,000",andrew r jassy,amazon
...,...,...,...,...
894,1930,"20,300",thomas sinnickson gayner,markel corporation
895,1939,"12,023",j powell brown cpcu cpcu,brown & brown inc
896,,"16,196",shingo konomoto,nomura research institute ltd
897,1986,"52,000",william joseph hornbuckle iv,mgm resorts international


In [663]:
match_join_df = join(matching_dataset_left, matching_dataset_right)

KeyError: "None of [Index(['name_l', 'name_r'], dtype='object')] are in the [columns]"

In [644]:
features_difference, feature_value_difference = get_features(difference_l, difference_r, 0.95, True)
pairs_difference = get_pairs(features_difference, feature_value_difference)

In [645]:
features_difference.sum(axis=1).value_counts().sort_index(ascending=False)

1.0      171
0.0    46341
dtype: int64

In [646]:
left = []
right = []

for elem in pairs_difference:
    left.append(difference_l.loc[elem[0]])
    right.append(difference_r.loc[elem[1]])

matching_difference_dataset_left = pd.DataFrame(left)
matching_difference_dataset_right = pd.DataFrame(right)
matching_difference_dataset_left = matching_difference_dataset_left.reset_index(drop=True)
matching_difference_dataset_right = matching_difference_dataset_right.reset_index(drop=True)

In [647]:
difference_join_df = join(matching_dataset_left, matching_dataset_right)
difference_join_df

KeyError: "None of [Index(['name_l', 'name_r'], dtype='object')] are in the [columns]"

In [591]:
result = pd.concat([match_join_df, difference_join_df])
result

,country,continent,market_cap,categories,founded,employees,ceo,name
0,cupertino,ca,2825000000000,"unitedstates, technology, consumerelectronics,...","april 1, 1976","100,000",timothy d cook,apple
1,washington,dc,208030000000,"unitedstates, healthcare, diagnostics&research...",1969,"78,000",rainer m blair,danaher
2,kenilworth,nj,210380000000,"unitedstates, healthcare, drugmanufacturersgen...",1891,"67,000",robert m davis jd,merck
3,melbourne,vic,a$281.66 billion,"australia, basicmaterials, otherindustrialmeta...",1885,"40,110",mike p henry bsc bsc chem,bhp
4,beaverton,or,213600000000,"unitedstates, consumerdiscretionary, footwear&...","january 25, 1964","73,300",john j donahoe ii,nike
...,...,...,...,...,...,...,...,...
166,riyadh,saudiarabia,sr76.56 billion,"saudiarabia, financials, banks, saudiarabiafin...",,"3,280",,alinma bank
167,london,unitedkingdom,15.84 billion,"unitedkingdom, financials, banks, unitedkingdo...","as chartered bank of india, australia, and chi...","81,770",,standard chartered plc
168,,,hk$165.86 billion,"hongkong, consumerstaples, packagedfoods, hong...",,,,china mengniu dairy co ltd
169,,,cn134.96 billion,"china, basicmaterials, otherindustrialmetals&m...",,,,china northern rare earth


In [596]:
unique_l = pd.concat([difference_l, matching_dataset_left]).drop_duplicates(keep=False)
unique_r = pd.concat([difference_r, matching_dataset_right]).drop_duplicates(keep=False)

In [597]:
unique_l

,name,country,continent,founded,employees,ceo,market_cap,categories,name_l,country_l,continent_l,founded_l,employees_l,ceo_l,market_cap_l,categories_l
15,icbc,beijing,china,,"439,787",,cn1.593 trillion,"china, financials, banks, chinafinancials, chi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,kweichow moutai,renhuai,china,,"29,031",,cn2.216 trillion,"china, consumerstaples, beverageswineries&dist...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,agricultural bank of china,beijing,china,,"459,000",,cn1.050 trillion,"china, financials, banks, chinafinancials, chi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,catl,ningde,china,,"33,078",,cn1.181 trillion,"china, industrials, electricalequipment&parts,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,china construction bank,beijing,china,,"373,814",,cn1.205 trillion,"china, financials, banks, chinafinancials, chi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,alinma bank,riyadh,saudiarabia,,"3,280",,sr76.56 billion,"saudiarabia, financials, banks, saudiarabiafin..."
167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,standard chartered plc,london,unitedkingdom,"as chartered bank of india, australia, and chi...","81,770",,15.84 billion,"unitedkingdom, financials, banks, unitedkingdo..."
168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,china mengniu dairy co ltd,,,,,,hk$165.86 billion,"hongkong, consumerstaples, packagedfoods, hong..."
169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,china northern rare earth,,,,,,cn134.96 billion,"china, basicmaterials, otherindustrialmetals&m..."


In [601]:
difference_l

,name,country,continent,founded,employees,ceo,market_cap,categories
15,icbc,beijing,china,,"439,787",,cn1.593 trillion,"china, financials, banks, chinafinancials, chi..."
27,kweichow moutai,renhuai,china,,"29,031",,cn2.216 trillion,"china, consumerstaples, beverageswineries&dist..."
57,agricultural bank of china,beijing,china,,"459,000",,cn1.050 trillion,"china, financials, banks, chinafinancials, chi..."
66,catl,ningde,china,,"33,078",,cn1.181 trillion,"china, industrials, electricalequipment&parts,..."
69,china construction bank,beijing,china,,"373,814",,cn1.205 trillion,"china, financials, banks, chinafinancials, chi..."
...,...,...,...,...,...,...,...,...
995,asahi group holdings ltd,tokyo,japan,,"30,020",atsushi katsuki,2.231 trillion,"japan, consumerstaples, beveragesbrewers, japa..."
996,haitong securities co ltd,shanghai,china,,"11,282",,cn115.98 billion,"china, financials, capitalmarkets, chinafinanc..."
997,catalent inc,somerset,nj,2007,"17,300",john r chiminski,18240000000,"unitedstates, healthcare, drugmanufacturersspe..."
998,quanta services inc,houston,tx,1997,"43,700",earl c austin jr,18250000000,"unitedstates, industrials, engineering&constru..."


In [602]:
matching_dataset_left

,name_l,country_l,continent_l,founded_l,employees_l,ceo_l,market_cap_l,categories_l
0,icbc,beijing,china,,"439,787",,cn1.593 trillion,"china, financials, banks, chinafinancials, chi..."
1,kweichow moutai,renhuai,china,,"29,031",,cn2.216 trillion,"china, consumerstaples, beverageswineries&dist..."
2,agricultural bank of china,beijing,china,,"459,000",,cn1.050 trillion,"china, financials, banks, chinafinancials, chi..."
3,catl,ningde,china,,"33,078",,cn1.181 trillion,"china, industrials, electricalequipment&parts,..."
4,china construction bank,beijing,china,,"373,814",,cn1.205 trillion,"china, financials, banks, chinafinancials, chi..."
...,...,...,...,...,...,...,...,...
166,alinma bank,riyadh,saudiarabia,,"3,280",,sr76.56 billion,"saudiarabia, financials, banks, saudiarabiafin..."
167,standard chartered plc,london,unitedkingdom,"as chartered bank of india, australia, and chi...","81,770",,15.84 billion,"unitedkingdom, financials, banks, unitedkingdo..."
168,china mengniu dairy co ltd,,,,,,hk$165.86 billion,"hongkong, consumerstaples, packagedfoods, hong..."
169,china northern rare earth,,,,,,cn134.96 billion,"china, basicmaterials, otherindustrialmetals&m..."


In [462]:
column_left, column_right = rename_columns(matching_dataset_left.columns.values.tolist(),
                                           matching_dataset_right.columns.values.tolist())
matching_dataset_left.columns = column_left
matching_dataset_right.columns = column_right

In [463]:
joined_df = pd.concat([matching_dataset_left, matching_dataset_right], axis=1)
joined_df

,name_l,country_l,continent_l,founded_l,employees_l,ceo_l,market_cap_l,categories_l,founded_r,employees_r,ceo_r,name_r
0,apple,cupertino,ca,"april 1, 1976","100,000",timothy d cook,2825000000000,"unitedstates, technology, consumerelectronics,...","april 1, 1976","100,000",timothy d cook,apple
1,danaher,washington,dc,1969,"78,000",rainer m blair,208030000000,"unitedstates, healthcare, diagnostics&research...",1969,"78,000",rainer m blair,danaher
2,merck,kenilworth,nj,1891,"67,000",robert m davis jd,210380000000,"unitedstates, healthcare, drugmanufacturersgen...",1891,"67,000",robert m davis jd,merck
3,bhp,melbourne,vic,1885,"40,110",mike p henry bsc bsc chem,a$281.66 billion,"australia, basicmaterials, otherindustrialmeta...",1885,"40,110",mike p henry bsc bsc chem,bhp
4,nike,beaverton,or,"january 25, 1964","73,300",john j donahoe ii,213600000000,"unitedstates, consumerdiscretionary, footwear&...","january 25, 1964","73,300",john j donahoe ii,nike
...,...,...,...,...,...,...,...,...,...,...,...,...
723,rede d'or sao luiz sa,saopaulo,sp,,,paulo junqueira moll,r$99.71 billion,"brazil, healthcare, medicalcarefacilities, bra...",,,paulo junqueira moll,rede d'or sao luiz sa
724,teck resources limited,vancouver,bc,,"10,600",donald r lindsay bsc honours mba,c$26.81 billion,"canada, basicmaterials, otherindustrialmetals&...",,"10,600",donald r lindsay bsc honours mba,teck resources limited
725,the saudi british bank,riyadh,saudiarabia,,"4,156",anthony william cripps,sr80.45 billion,"saudiarabia, financials, banks, saudiarabiafin...",,"4,156",anthony william cripps,the saudi british bank
726,mitsui fudosan co ltd,tokyo,japan,,"23,992",masanobu komoda,2.629 trillion,"japan, realestate, realestatediversified, japa...",,"23,992",masanobu komoda,mitsui fudosan co ltd


In [464]:
duplicates = find_duplicated_columns(df_l.columns.values.tolist(), df_r.columns.values.tolist())
duplicates

['name', 'founded', 'employees', 'ceo']

In [465]:
for col in duplicates:
    candidates = joined_df[[col + '_l', col + '_r']].sample(20)
    candidate_labels = candidates.columns.values.tolist()
    display(candidates.sample(10))
    time.sleep(1)
    idx_drop = int(input("Inserisci l'indice (0 o 1) della colonna che vuoi scartare nel dataset finale: "))
    joined_df.drop(candidate_labels[idx_drop], axis=1, inplace=True)


,name_l,name_r
649,zebra technologies corporation,zebra technologies corporation
657,geberit ag,geberit ag
377,simon property group inc,simon property group inc
504,naturgy energy group sa,naturgy energy group sa
565,caixabank sa,caixabank sa
257,marvell technology group ltd,marvell technology group ltd
479,oneok inc,oneok inc
609,mid-america apartment communities inc,mid-america apartment communities inc
562,sartorius,sartorius
587,albemarle corporation,albemarle corporation


,founded_l,founded_r
361,1883,1883
471,,
662,,
460,1995,1995
242,,
511,,
424,1969,1969
478,1989,1989
627,,
655,1946,1946


,employees_l,employees_r
311,"47,099","47,099"
459,"52,450","52,450"
608,"316,078","316,078"
151,"27,605","27,605"
717,"2,742","2,742"
219,"50,000","50,000"
365,"77,958","77,958"
272,"47,000","47,000"
541,"26,000","26,000"
560,"22,700","22,700"


IndexError: list index out of range

In [435]:
joined_df.columns = [col[:-2] for col in joined_df.columns.values.tolist()]
joined_df

,country,continent,market_cap,categories,founded,employees,ceo,name
0,cupertino,ca,2825000000000,"unitedstates, technology, consumerelectronics,...","april 1, 1976","100,000",timothy d cook,apple
1,washington,dc,208030000000,"unitedstates, healthcare, diagnostics&research...",1969,"78,000",rainer m blair,danaher
2,kenilworth,nj,210380000000,"unitedstates, healthcare, drugmanufacturersgen...",1891,"67,000",robert m davis jd,merck
3,melbourne,vic,a$281.66 billion,"australia, basicmaterials, otherindustrialmeta...",1885,"40,110",mike p henry bsc bsc chem,bhp
4,beaverton,or,213600000000,"unitedstates, consumerdiscretionary, footwear&...","january 25, 1964","73,300",john j donahoe ii,nike
...,...,...,...,...,...,...,...,...
647,saopaulo,sp,r$99.71 billion,"brazil, healthcare, medicalcarefacilities, bra...",,,paulo junqueira moll,rede d'or sao luiz sa
648,vancouver,bc,c$26.81 billion,"canada, basicmaterials, otherindustrialmetals&...",,"10,600",donald r lindsay bsc honours mba,teck resources limited
649,riyadh,saudiarabia,sr80.45 billion,"saudiarabia, financials, banks, saudiarabiafin...",,"4,156",anthony william cripps,the saudi british bank
650,tokyo,japan,2.629 trillion,"japan, realestate, realestatediversified, japa...",,"23,992",masanobu komoda,mitsui fudosan co ltd


problema con record che hanno valori vuoti

In [506]:
df_r.loc[df_r['name'] == 'abb ltd']

,founded,employees,ceo,name
229,1988,"104,400",,abb ltd


In [507]:
df_l.loc[df_l['name'] == 'abb ltd']

,name,country,continent,founded,employees,ceo,market_cap,categories
220,abb ltd,zurich,switzerland,1988,"104,400",,chf62.20 billion,"switzerland, industrials, electricalequipment&..."


In [510]:
df_r['name'][229] == df_l['name'][220]

True

In [512]:
df_r['ceo'][229] == df_l['ceo'][220]

True

In [533]:
empty_l = difference_l.loc[difference_l['ceo'] == '']
empty_l

,name,country,continent,founded,employees,ceo,market_cap,categories
15,icbc,beijing,china,,"439,787",,cn1.593 trillion,"china, financials, banks, chinafinancials, chi..."
27,kweichow moutai,renhuai,china,,"29,031",,cn2.216 trillion,"china, consumerstaples, beverageswineries&dist..."
57,agricultural bank of china,beijing,china,,"459,000",,cn1.050 trillion,"china, financials, banks, chinafinancials, chi..."
66,catl,ningde,china,,"33,078",,cn1.181 trillion,"china, industrials, electricalequipment&parts,..."
69,china construction bank,beijing,china,,"373,814",,cn1.205 trillion,"china, financials, banks, chinafinancials, chi..."
...,...,...,...,...,...,...,...,...
976,astra international tbk pt,jakarta,indonesia,,"123,894",,17.11 billion,"indonesia, consumerdiscretionary, autoparts, i..."
984,nice ltd,raaanana,israel,1986,"6,800",,594.2 million,"israel, technology, softwareapplication, israe..."
987,zhejiang huayou cobalt co ltd,,,,,,cn122.16 billion,"china, basicmaterials, otherindustrialmetals&m..."
993,restaurant brands international inc,toronto,on,"december 15, 2014","5,700",,c$22.78 billion,"canada, consumerdiscretionary, restaurants, ca..."


In [534]:
empty_r = difference_r.loc[difference_r['ceo'] == '']
empty_r

,founded,employees,ceo,name
23,,"29,031",,kweichow moutai
35,,"439,787",,icbc
51,march 1996,"104,323",,novartis
58,,"373,814",,china construction bank
61,,"33,078",,catl
...,...,...,...,...
877,"as chartered bank of india, australia, and c...","81,770",,standard chartered plc
888,,"3,280",,alinma bank
891,,"15,198",,guotai junan securities co ltd
892,,"27,318",,nippon paint holdings co ltd


In [504]:
difference_l['name'][220] == difference_r['name'][220]

KeyError: 220

In [540]:
features_empty, feature_value_empty = get_features(difference_l, difference_r, 0.95, True)
pairs_empty = get_pairs(features_empty, feature_value_empty)

In [541]:
feature_value_empty

0

In [543]:
difference_l['name'][15]

'icbc'

In [544]:
difference_r['name'][35]

'icbc'

In [542]:
features_empty

name
15  23    0.0
    35    1.0
    51    0.0
    58    0.0
    61    0.0
...       ...
999 877   0.0
    888   0.0
    891   0.0
    892   0.0
    898   0.0

[46512 rows x 1 columns]

In [545]:
features_empty.sum(axis=1).value_counts().sort_index(ascending=False)

1.0      171
0.0    46341
dtype: int64